In [1]:
BRANCH='main'

In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell

# install NeMo
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[nlp]

In [2]:
from nemo.collections import nlp as nemo_nlp
from nemo.utils.exp_manager import exp_manager

import os
import wget 
import torch
import pytorch_lightning as pl
from omegaconf import OmegaConf

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

[NeMo W 2022-01-21 13:35:51 experimental:27] Module <function get_argmin_mat at 0x7fb6b7edc820> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-01-21 13:35:51 experimental:27] Module <function getMultiScaleCosAffinityMatrix at 0x7fb6d007c550> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-01-21 13:35:51 experimental:27] Module <function parse_scale_configs at 0x7fb6b7ee31f0> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022

In this tutorial, we are going to describe how to finetune BioMegatron - a [BERT](https://arxiv.org/abs/1810.04805)-like [Megatron-LM](https://arxiv.org/pdf/1909.08053.pdf) model pre-trained on large biomedical text corpus ([PubMed](https://pubmed.ncbi.nlm.nih.gov/) abstracts and full-text commercial use collection) - on the [NCBI Disease Dataset](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3951655/) for Named Entity Recognition.

The model size of Megatron-LM can be larger than BERT, up to multi-billion parameters, compared to 345 million parameters of BERT-large.
There are some alternatives of BioMegatron, most notably [BioBERT](https://arxiv.org/abs/1901.08746). Compared to BioBERT BioMegatron is larger by model size and pre-trained on larger text corpus.

A more general tutorial of using BERT-based models, including Megatron-LM, for downstream natural language processing tasks can be found [here](https://github.com/NVIDIA/NeMo/blob/stable/tutorials/nlp/01_Pretrained_Language_Models_for_Downstream_Tasks.ipynb).

# Task Description
**Named entity recognition (NER)**, also referred to as entity chunking, identification or extraction, is the task of detecting and classifying key information (entities) in text.

For instance, **given sentences from medical abstracts, what diseases are mentioned?**<br>
In this case, our data input is sentences from the abstracts, and our labels are the precise locations of the named disease entities.  Take a look at the information provided for the dataset.

For more details and general examples on Named Entity Recognition, please refer to the [Token Classification and Named Entity Recognition tutorial notebook](https://github.com/NVIDIA/NeMo/blob/stable/tutorials/nlp/Token_Classification_Named_Entity_Recognition.ipynb).

# Dataset

The [NCBI-disease corpus](https://www.ncbi.nlm.nih.gov/CBBresearch/Dogan/DISEASE/) is a set of 793 PubMed abstracts, annotated by 14 annotators. The annotations take the form of HTML-style tags inserted into the abstract text using the clearly defined rules.  The annotations identify named diseases, and can be used to fine-tune a language model to identify disease mentions in future abstracts, *whether those diseases were part of the original training set or not*.

Here's an example of what an annotated abstract from the corpus looks like:

```html
10021369	Identification of APC2, a homologue of the <category="Modifier">adenomatous polyposis coli tumour</category> suppressor .	The <category="Modifier">adenomatous polyposis coli ( APC ) tumour</category>-suppressor protein controls the Wnt signalling pathway by forming a complex with glycogen synthase kinase 3beta ( GSK-3beta ) , axin / conductin and betacatenin . Complex formation induces the rapid degradation of betacatenin . In <category="Modifier">colon carcinoma</category> cells , loss of APC leads to the accumulation of betacatenin in the nucleus , where it binds to and activates the Tcf-4 transcription factor ( reviewed in [ 1 ] [ 2 ] ) . Here , we report the identification and genomic structure of APC homologues . Mammalian APC2 , which closely resembles APC in overall domain structure , was functionally analyzed and shown to contain two SAMP domains , both of which are required for binding to conductin . Like APC , APC2 regulates the formation of active betacatenin-Tcf complexes , as demonstrated using transient transcriptional activation assays in APC - / - <category="Modifier">colon carcinoma</category> cells . Human APC2 maps to chromosome 19p13 . 3 . APC and APC2 may therefore have comparable functions in development and <category="SpecificDisease">cancer</category> .
```

In this example, we see the following tags within the abstract:
```html
<category="Modifier">adenomatous polyposis coli tumour</category>
<category="Modifier">adenomatous polyposis coli ( APC ) tumour</category>
<category="Modifier">colon carcinoma</category>
<category="Modifier">colon carcinoma</category>
<category="SpecificDisease">cancer</category>
```

For our purposes, we will consider any identified category (such as "Modifier", "Specific Disease", and a few others) to generally be a "disease".

Let's download the dataset.

In [3]:
DATA_DIR = "DATA_DIR"
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(os.path.join(DATA_DIR, 'SA'), exist_ok=True)

## Downloading Financial Phrase Bank Dataset

The datase is collected by Malo et al. 2014, and can be downloaded from this [link](https://www.researchgate.net/profile/Pekka_Malo/publication/251231364_FinancialPhraseBank-v10/data/0c96051eee4fb1d56e000000/FinancialPhraseBank-v10.zip). The zip file for the Financial Phrase Bank Dataset has been provided for ease of download and use.

In [4]:
!wget https://www.researchgate.net/profile/Pekka_Malo/publication/251231364_FinancialPhraseBank-v10/data/0c96051eee4fb1d56e000000/FinancialPhraseBank-v10.zip
!mv FinancialPhraseBank-v10.zip {DATA_DIR}
!unzip -f {DATA_DIR}/FinancialPhraseBank-v10.zip -d {DATA_DIR}

--2022-01-20 01:48:29--  https://www.researchgate.net/profile/Pekka_Malo/publication/251231364_FinancialPhraseBank-v10/data/0c96051eee4fb1d56e000000/FinancialPhraseBank-v10.zip
Resolving www.researchgate.net (www.researchgate.net)... 104.17.32.105, 104.17.33.105, 2606:4700::6811:2069, ...
Connecting to www.researchgate.net (www.researchgate.net)|104.17.32.105|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.researchgate.net/profile/Pekka-Malo/publication/251231364_FinancialPhraseBank-v10/data/0c96051eee4fb1d56e000000/FinancialPhraseBank-v10.zip [following]
--2022-01-20 01:48:29--  https://www.researchgate.net/profile/Pekka-Malo/publication/251231364_FinancialPhraseBank-v10/data/0c96051eee4fb1d56e000000/FinancialPhraseBank-v10.zip
Reusing existing connection to www.researchgate.net:443.
HTTP request sent, awaiting response... 200 OK
Length: 681890 (666K) [application/zip]
Saving to: ‘FinancialPhraseBank-v10.zip’

FinancialPhraseBank 

In [5]:
# If you want to see more examples, you can explore the text of the corpus using the file browser to the left, or open files directly, for example typing a command like the following in a code-cell:

! head -1 $DATA_DIR/FinancialPhraseBank-v1.0/Sentences_50Agree.txt

According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .@neutral


We have two datasets derived from this corpus:  a text classification dataset and a named entity recognition (NER) dataset.  The text classification dataset labels the abstracts among three broad disease groupings.  We'll use this simple split to demonstrate the NLP text classification task.   The NER dataset labels individual words as diseases.  This dataset will be used for the NLP NER task.  

## Pre-process dataset
A pre-processed NCBI-disease dataset for NER can be found [here](https://github.com/spyysalo/ncbi-disease/tree/master/conll) or [here](https://github.com/dmis-lab/biobert#datasets).<br>
We download the files under {DATA_DIR/NER} directory.

In [6]:
import json
import random

random.seed(1234)
files = ['Sentences_50Agree.txt', 'Sentences_66Agree.txt', 'Sentences_75Agree.txt', 'Sentences_AllAgree.txt']
base_dir = DATA_DIR + '/FinancialPhraseBank-v1.0/'
files = [base_dir + f for f in files]

alllines = []
for fn in files:
    with open(fn, 'r', encoding="ISO-8859-1") as f:
        alllines.extend(f.readlines())

random.shuffle(alllines)
fold = 10
fold_size = len(alllines) // fold

chunk_start = list(range(0, 14780, 1478))

chunks = []

for start_id in chunk_start:
    chunks.append(alllines[start_id:start_id+fold_size])

special = '<|endoftext|>'

def gen_file(data, fold_id, split_type):
    filename = "{}/{}_{}.txt".format(base_dir, split_type, fold_id)
    with open(filename, 'w') as f:
        obj = {}
        for line in data:
            splits = line.split('@')
            part1 = splits[0].strip()
            part2 = splits[1].strip()
            obj['sentence'] = part1 +' Sentiment '
            obj['sentiment'] = part2
            f.write(json.dumps(obj)+'\n')


def gen_fold(fold_number):
    lists = list(range(fold))
    test_id = (fold_number + fold) % fold
    val_id = (fold_number + fold - 1) % fold
    test_set = chunks[test_id]
    val_set = chunks[val_id]
    lists.remove(test_id)
    lists.remove(val_id)
    train_set = []
    for idd in lists:
        train_set += chunks[idd]
    gen_file(train_set, fold_number, 'train')
    gen_file(val_set, fold_number, 'validation')
    gen_file(test_set, fold_number, 'test')

for i in range(fold):
    gen_fold(i)

The NER task requires two files: the text sentences, and the labels.  Run the next two cells to see a sample of the two files.

In [9]:
!head -n 2 $DATA_DIR/FinancialPhraseBank-v1.0/train_0.txt

{"sentence": "The contract includes heating plant equipment and associated installation work . Sentiment ", "sentiment": "neutral"}
{"sentence": "The utility will also provide services related to electricity management , such as hedging trades and risk management and reporting . Sentiment ", "sentiment": "neutral"}


## Convert the Megatron-LM Weights to Nemo file

P-Tuning method works the best with large GPT lanague models. From our experiences, models of size 5B or above give good performance. If you already have a large GPT model ready, skip this section. 

In this example, we will use the pretrained 344M NeMo Megatron GPT model from [Megatron-LM project](https://github.com/NVIDIA/Megatron-LM). To load it in NeMo Megatron, We first need to convert the Megatron-LM checkpoint to the `.nemo` file. Let's download the pretrained model weights and vocabulary file.



In [15]:
import pathlib
gpt_file = 'megatron_lm_345m_v0.0.zip'
vocab_file = 'gpt2-vocab.json'
merge_file = 'gpt2-merge.txt'
checkpoint_filename = 'model_optim_rng.pt'

if not pathlib.Path(gpt_file).exists():
    !wget --content-disposition https://api.ngc.nvidia.com/v2/models/nvidia/megatron_lm_345m/versions/v0.0/zip -O $gpt_file
    !unzip -f $gpt_file
    !wget https://s3.amazonaws.com/models.huggingface.co/bert/$vocab_file -O $vocab_file 
    !wget https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt -O $merge_file



In [4]:
WORK_DIR = "WORK_DIR"
os.makedirs(WORK_DIR, exist_ok=True)

# Prepare the model parameters 
# download the model's configuration file 
config_dir = WORK_DIR + '/configs/'
MODEL_CONFIG = "megatron_gpt_config.yaml"
os.makedirs(config_dir, exist_ok=True)
if not os.path.exists(config_dir + MODEL_CONFIG):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/language_modeling/conf/' + MODEL_CONFIG, config_dir)
else:
    print ('config file is already exists')

config file is already exists


In [6]:
# this line will print the entire config of the model
config_path = f'{WORK_DIR}/configs/{MODEL_CONFIG}'
print(config_path)
config = OmegaConf.load(config_path)
config.model.num_layers = 24
config.model.hidden_size = 1024
config.model.ffn_hidden_size = 4096
config.model.num_attention_heads = 16
config.model.tokenizer.vocab_file = vocab_file
config.model.tokenizer.merge_file = merge_file
config.model.tensor_model_parallel_size = 1
config.model.data.data_prefix = ''
config.model.max_position_embeddings = 1024
config.model.data.seq_length = 1024
config.model.encoder_seq_length = 1024
config.cfg = {}
config.cfg.cfg = config.model
with open('hparams.yaml', 'w') as f:
    f.write(OmegaConf.to_yaml(config.cfg))

WORK_DIR/configs/megatron_gpt_config.yaml


In [7]:
import os
PWD = os.getcwd()
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/language_modeling/megatron_lm_ckpt_to_nemo.py')
!python -m torch.distributed.run --nproc_per_node=1 megatron_lm_ckpt_to_nemo.py --checkpoint_folder=$PWD/release/mp_rank_00/ --checkpoint_name=$checkpoint_filename --hparams_file=$PWD/hparams.yaml --nemo_file_path=$PWD/gpt_344m.nemo --model_type=gpt --tensor_model_parallel_size=1

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

[NeMo W 2022-01-20 21:01:09 experimental:27] Module <function get_argmin_mat at 0x7fd05070be50> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-01-20 21:01:09 experimental:27] Module <function getMultiScaleCosAffinityMatrix at 0x7fd05070bee0> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-01-20 21:01:09 experimental:27] Module <function parse_scale_configs at 0x7fd050717940> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022

# Model configuration

Our Named Entity Recognition model is comprised of the pretrained [BERT](https://arxiv.org/pdf/1810.04805.pdf) model followed by a Token Classification layer.

The model is defined in a config file which declares multiple important sections. They are:
- **model**: All arguments that are related to the Model - language model, token classifier, optimizer and schedulers, datasets and any other related information

- **trainer**: Any argument to be passed to PyTorch Lightning

In [5]:
MODEL_CONFIG = "ptune_text_classification_config.yaml"

In [6]:
# download the model's configuration file 
config_dir = WORK_DIR + '/configs/'
os.makedirs(config_dir, exist_ok=True)
if not os.path.exists(config_dir + MODEL_CONFIG):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/token_classification/conf/' + MODEL_CONFIG, config_dir)
else:
    print ('config file is already exists')

config file is already exists


In [7]:
# this line will print the entire config of the model
config_path = f'{WORK_DIR}/configs/{MODEL_CONFIG}'
print(config_path)
config = OmegaConf.load(config_path)
# Note: these are small batch-sizes - increase as appropriate to available GPU capacity
config.model.train_ds.batch_size=8
config.model.validation_ds.batch_size=8

WORK_DIR/configs/ptune_text_classification_config.yaml


In [8]:
# in this tutorial train and dev datasets are located in the same folder, so it is enought to add the path of the data directory to the config
#config.model.dataset.classes = ['positive', 'neutral', 'negative']
config.model.train_ds.file_path = DATA_DIR+'/FinancialPhraseBank-v1.0/train_0.txt'
config.model.validation_ds.file_path = DATA_DIR+'/FinancialPhraseBank-v1.0/validation_0.txt'
config.model.test_ds.file_path = DATA_DIR+'/FinancialPhraseBank-v1.0/test_0.txt'


# if you want to decrease the size of your datasets, uncomment the lines below:
# NUM_SAMPLES = 1000
# config.model.train_ds.num_samples = NUM_SAMPLES
# config.model.validation_ds.num_samples = NUM_SAMPLES

In [9]:
print(OmegaConf.to_yaml(config))

trainer:
  gpus: 1
  num_nodes: 1
  max_epochs: 100
  max_steps: null
  accumulate_grad_batches: 1
  gradient_clip_val: 0.0
  precision: 32
  accelerator: ddp
  log_every_n_steps: 1
  val_check_interval: 1.0
  resume_from_checkpoint: null
  num_sanity_val_steps: 0
  checkpoint_callback: false
  logger: false
model:
  tensor_model_parallel_size: 2
  seed: 1234
  nemo_path: ptune_text_classification_model.nemo
  use_lm_finetune: false
  pseudo_token: '[PROMPT]'
  tokenizer:
    library: megatron
    type: GPT2BPETokenizer
    model: null
    vocab_file: null
    merge_file: null
  language_model:
    nemo_file: null
  prompt_encoder:
    template:
    - 3
    - 3
    - 0
    dropout: 0.0
  dataset:
    classes: ???
  train_ds:
    file_path: DATA_DIR/FinancialPhraseBank-v1.0/train_0.txt
    batch_size: 8
    shuffle: true
    num_samples: -1
    num_workers: 3
    drop_last: false
    pin_memory: false
  validation_ds:
    file_path: DATA_DIR/FinancialPhraseBank-v1.0/validation_0.txt
   

# Model Training
## Setting up Data within the config

Among other things, the config file contains dictionaries called dataset, train_ds and validation_ds. These are configurations used to setup the Dataset and DataLoaders of the corresponding config.



We assume that both training and evaluation files are located in the same directory, and use the default names mentioned during the data download step. 
So, to start model training, we simply need to specify `model.dataset.data_dir`, like we are going to do below.



Also notice that some config lines, including `model.dataset.data_dir`, have `???` in place of paths, this means that values for these fields are required to be specified by the user.

Let's now add the data directory path to the config.

In [10]:
# in this tutorial train and dev datasets are located in the same folder, so it is enought to add the path of the data directory to the config
config.model.dataset.data_dir = os.path.join(DATA_DIR, 'SA')

# if you want to decrease the size of your datasets, uncomment the lines below:
# NUM_SAMPLES = 1000
# config.model.train_ds.num_samples = NUM_SAMPLES
# config.model.validation_ds.num_samples = NUM_SAMPLES

## Building the PyTorch Lightning Trainer

NeMo models are primarily PyTorch Lightning modules - and therefore are entirely compatible with the PyTorch Lightning ecosystem.

Let's first instantiate a Trainer object

In [11]:
print("Trainer config - \n")
print(OmegaConf.to_yaml(config.trainer))

Trainer config - 

gpus: 1
num_nodes: 1
max_epochs: 100
max_steps: null
accumulate_grad_batches: 1
gradient_clip_val: 0.0
precision: 32
accelerator: ddp
log_every_n_steps: 1
val_check_interval: 1.0
resume_from_checkpoint: null
num_sanity_val_steps: 0
checkpoint_callback: false
logger: false



In [12]:
from nemo.collections.nlp.parts.nlp_overrides import NLPDDPPlugin


# lets modify some trainer configs
# checks if we have GPU available and uses it
cuda = 1 if torch.cuda.is_available() else 0
config.trainer.gpus = cuda

# for PyTorch Native AMP set precision=16
config.trainer.precision = 16 if torch.cuda.is_available() else 32

# remove distributed training flags
config.trainer.accelerator = None

trainer = pl.Trainer(plugins=[NLPDDPPlugin()], **config.trainer)

[NeMo W 2022-01-21 13:37:28 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/plugins/training_type/ddp.py:107: LightningDeprecationWarning: Argument `num_nodes` in `DDPPlugin` is deprecated in v1.4, and will be removed in v1.6. Notice that it will be overriden by the trainer setting.
      rank_zero_deprecation(
    
[NeMo W 2022-01-21 13:37:28 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/plugins/training_type/ddp.py:113: LightningDeprecationWarning: Argument `sync_batchnorm` in `DDPPlugin` is deprecated in v1.4, and will be removed in v1.6. Notice that it will be overriden by the trainer setting.
      rank_zero_deprecation(
    
[NeMo W 2022-01-21 13:37:28 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:324: LightningDeprecationWarning: Passing <nemo.collections.nlp.parts.nlp_overrides.NLPDDPPlugin object at 0x7fb74a3b61f0> `strategy` to the `plugins` flag in

## Setting up a NeMo Experiment

NeMo has an experiment manager that handles logging and checkpointing for us, so let's use it:

In [13]:
exp_dir = exp_manager(trainer, config.get("exp_manager", None))
os.makedirs(WORK_DIR, exist_ok=True)

# the exp_dir provides a path to the current experiment for easy access
exp_dir = str(exp_dir)
exp_dir

[NeMo I 2022-01-21 13:37:41 exp_manager:283] Experiments will be logged at /NeMo/tutorials/nlp/nemo_experiments/PTuneTextClassification/2022-01-21_13-37-41
[NeMo I 2022-01-21 13:37:41 exp_manager:648] TensorboardLogger has been set up


[NeMo W 2022-01-21 13:37:41 exp_manager:889] The checkpoint callback was told to monitor a validation value and trainer's max_steps was set to -1. Please ensure that max_steps will run for at least 1 epochs to ensure that checkpointing will not error out.
[NeMo W 2022-01-21 13:37:41 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:243: LightningDeprecationWarning: `ModelCheckpoint(every_n_val_epochs)` is deprecated in v1.4 and will be removed in v1.6. Please use `every_n_epochs` instead.
      rank_zero_deprecation(
    


'/NeMo/tutorials/nlp/nemo_experiments/PTuneTextClassification/2022-01-21_13-37-41'

To load the pretrained BERT LM model, we can either load it from the converted `.nemo` file as shown above or load it from a list of included model names. 

We can get the list of names by following command 
```python
# complete list of supported BERT-like models
print(nemo_nlp.modules.get_pretrained_lm_models_list())
```
We can change the `model.language_mode` config to use it
```python
# add the specified above model parameters to the config
config.model.language_model.pretrained_model_name = MODEL_NAME
```

In this notebook, we will use the converted `.nemo` file as our LM model, which is BioMegatron, [Megatron-LM BERT](https://arxiv.org/abs/1909.08053) pre-trained on [PubMed](https://pubmed.ncbi.nlm.nih.gov/) biomedical text corpus.

In [16]:
# add the specified above model parameters to the config
# config.model.language_model.pretrained_model_name = PRETRAINED_BERT_MODEL
config.model.language_model.nemo_file = 'gpt_344m.nemo'
config.model.tensor_model_parallel_size = 1
config.model.dataset.classes = ['positive', 'neutral', 'negative']
config.model.tokenizer.vocab_file = vocab_file
config.model.tokenizer.merge_file = merge_file

Now, we are ready to initialize our model. During the model initialization call, the dataset and data loaders we'll be prepared for training and evaluation.
Also, the pretrained BERT model will be downloaded, note it can take up to a few minutes depending on the size of the chosen BERT model.

In [17]:
from nemo.collections.nlp.models.text_classification.ptune_text_classification_model import PTuneTextClassificationModel
model_ptune = PTuneTextClassificationModel(cfg=config.model, trainer=trainer)

[NeMo I 2022-01-21 13:38:19 tokenizer_utils:190] Getting Megatron tokenizer for pretrained model name: megatron-gpt-345m and custom vocab file: /NeMo/tutorials/nlp/gpt2-vocab.json
[NeMo I 2022-01-21 13:38:19 tokenizer_utils:123] Getting HuggingFace AutoTokenizer with pretrained_model_name: gpt2, vocab_file: /NeMo/tutorials/nlp/gpt2-vocab.json, special_tokens_dict: {}, and use_fast: False


Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using mask_token, but it is not set yet.


[NeMo I 2022-01-21 13:38:34 tokenizer_utils:190] Getting Megatron tokenizer for pretrained model name: megatron-gpt-345m and custom vocab file: /tmp/tmp1vxu9jzs/3f23abcf03b94354899f3c5b5beab943_gpt2-vocab.json
[NeMo I 2022-01-21 13:38:34 tokenizer_utils:123] Getting HuggingFace AutoTokenizer with pretrained_model_name: gpt2, vocab_file: /tmp/tmp1vxu9jzs/3f23abcf03b94354899f3c5b5beab943_gpt2-vocab.json, special_tokens_dict: {}, and use_fast: False


Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using mask_token, but it is not set yet.


[NeMo I 2022-01-21 13:38:38 megatron_gpt_model:754] Padded vocab_size: 50304, original vocab_size: 50257, dummy tokens: 47.
[NeMo I 2022-01-21 13:38:39 tokenizer_utils:190] Getting Megatron tokenizer for pretrained model name: megatron-gpt-345m and custom vocab file: /tmp/tmp1vxu9jzs/3f23abcf03b94354899f3c5b5beab943_gpt2-vocab.json
[NeMo I 2022-01-21 13:38:39 tokenizer_utils:123] Getting HuggingFace AutoTokenizer with pretrained_model_name: gpt2, vocab_file: /tmp/tmp1vxu9jzs/3f23abcf03b94354899f3c5b5beab943_gpt2-vocab.json, special_tokens_dict: {}, and use_fast: False


Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using mask_token, but it is not set yet.


[NeMo I 2022-01-21 13:38:43 megatron_gpt_model:754] Padded vocab_size: 50304, original vocab_size: 50257, dummy tokens: 47.
[NeMo I 2022-01-21 13:38:43 save_restore_connector:149] Model MegatronGPTModel was successfully restored from /NeMo/tutorials/nlp/gpt_344m.nemo.
[NeMo I 2022-01-21 13:38:44 auto_tokenizer:171] 1 special tokens added, resize your model accordingly.


Using pad_token, but it is not set yet.
Using mask_token, but it is not set yet.


## Monitoring training progress
Optionally, you can create a Tensorboard visualization to monitor training progress.
If you're not using Colab, refer to [https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks](https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks) if you're facing issues with running the cell below.

In [18]:
try:
    from google import colab
    COLAB_ENV = True
except (ImportError, ModuleNotFoundError):
    COLAB_ENV = False

# Load the TensorBoard notebook extension
if COLAB_ENV:
    %load_ext tensorboard
    %tensorboard --logdir {exp_dir}
else:
    print("To use tensorboard, please use this notebook in a Google Colab environment.")

To use tensorboard, please use this notebook in a Google Colab environment.


In [19]:
# start model training
trainer.fit(model_ptune)

[NeMo W 2022-01-21 13:38:50 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:287: LightningDeprecationWarning: Base `Callback.on_train_batch_start` hook signature has changed in v1.5. The `dataloader_idx` argument will be removed in v1.7.
      rank_zero_deprecation(
    
[NeMo W 2022-01-21 13:38:50 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:287: LightningDeprecationWarning: Base `Callback.on_train_batch_end` hook signature has changed in v1.5. The `dataloader_idx` argument will be removed in v1.7.
      rank_zero_deprecation(
    
initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/1
I0121 13:38:50.209178 140425413850944 distributed_c10d.py:218] Added key: store_based_barrier_key:1 to store for rank: 0
I0121 13:38:50.209801 140425413850944 distributed_c10d.py:252] Rank 0: Completed store-based barrier for key:store_based_barrier_key:1 with 1 nodes.
----------

> initializing tensor model parallel with size 1
> initializing pipeline model parallel with size 1
> initializing data parallel with size 1
[NeMo I 2022-01-21 13:38:50 nlp_overrides:137] mp_rank: 0
[NeMo I 2022-01-21 13:38:50 nlp_overrides:138] dp_rank: 0


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4]
[NeMo W 2022-01-21 13:38:50 modelPT:475] The lightning trainer received accelerator: <pytorch_lightning.accelerators.gpu.GPUAccelerator object at 0x7fb6b0b60190>. We recommend to use 'ddp' instead.


[NeMo I 2022-01-21 13:38:50 modelPT:566] Optimizer config = Adam (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.999]
        eps: 1e-08
        lr: 1e-05
        weight_decay: 0.0005
    )
[NeMo I 2022-01-21 13:38:50 lr_scheduler:833] Scheduler "<nemo.core.optim.lr_scheduler.WarmupAnnealing object at 0x7fb67e23e2b0>" 
    will be used during training (effective maximum steps = 147800) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: 0.1
    last_epoch: -1
    max_steps: 147800
    )
[NeMo I 2022-01-21 13:38:51 nlp_overrides:92] Configuring DDP for model parallelism.



  | Name                  | Type                   | Params
-----------------------------------------------------------------
0 | model                 | MegatronGPTModel       | 354 M 
1 | embeddings            | VocabParallelEmbedding | 51.5 M
2 | classification_report | ClassificationReport   | 0     
3 | prompt_encoder        | PromptEncoder          | 14.7 M
-----------------------------------------------------------------
14.7 M    Trainable params
354 M     Non-trainable params
369 M     Total params
739.152   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

[NeMo W 2022-01-21 13:38:52 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
      warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
    
I0121 13:38:52.474958 140425413850944 distributed.py:902] Reducer buckets have been rebuilt in this iteration.


Validating: 0it [00:00, ?it/s]

[NeMo I 2022-01-21 13:41:51 ptune_text_classification_model:312] val_report: 
    label                                                precision    recall       f1           support   
    positive (label_id: 0)                                  49.78      27.68      35.58        401
    neutral (label_id: 1)                                   67.28      94.38      78.56        889
    negative (label_id: 2)                                  75.00       3.19       6.12        188
    -------------------
    micro avg                                               64.68      64.68      64.68       1478
    macro avg                                               64.02      41.75      40.09       1478
    weighted avg                                            63.51      64.68      57.68       1478
    


Epoch 0, global step 1477: val_loss reached 0.93801 (best 0.93801), saving model to "/NeMo/tutorials/nlp/nemo_experiments/PTuneTextClassification/2022-01-21_13-37-41/checkpoints/PTuneTextClassification--val_loss=0.9380-epoch=0.ckpt" as top 3
[NeMo W 2022-01-21 13:42:29 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:685: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
      rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
    


# Inference

To see how the model performs, we can run generate prediction similar to the way we did it earlier

In [ ]:
# let's first create a subset of our dev data
! head -n 100 $NER_DATA_DIR/text_dev.txt > $NER_DATA_DIR/sample_text_dev.txt
! head -n 100 $NER_DATA_DIR/labels_dev.txt > $NER_DATA_DIR/sample_labels_dev.txt

Now, let's generate predictions for the provided text file.
If labels file is also specified, the model will evaluate the predictions and plot confusion matrix. 

In [ ]:
model_ner.half().evaluate_from_file(
    text_file=os.path.join(NER_DATA_DIR, 'sample_text_dev.txt'),
    labels_file=os.path.join(NER_DATA_DIR, 'sample_labels_dev.txt'),
    output_dir=exp_dir,
    add_confusion_matrix=False,
    normalize_confusion_matrix=True,
    batch_size=1
)
# Please check matplotlib version if encountering any error plotting confusion matrix:
# https://stackoverflow.com/questions/63212347/importerror-cannot-import-name-png-from-matplotlib

## Training Script

If you have NeMo installed locally, you can also train the model with `nlp/token_classification/token_classification_train.py.`

To run training script, use:

`python token_classification_train.py model.dataset.data_dir=PATH_TO_DATA_DIR exp_manager.exp_dir=EXP_DIR model.language_model.pretrained_model_name=megatron-bert-cased model.tokenizer.vocab_file=VOCAB_FILE model.tokenizer.tokenizer_model=BertWordPieceCase model.language_model.nemo_file=NEMO_FILE`


The training could take several minutes and the result should look something like
```
[NeMo I 2020-05-22 17:13:48 token_classification_callback:82] Accuracy: 0.9882348032875798
[NeMo I 2020-05-22 17:13:48 token_classification_callback:86] F1 weighted: 98.82
[NeMo I 2020-05-22 17:13:48 token_classification_callback:86] F1 macro: 93.74
[NeMo I 2020-05-22 17:13:48 token_classification_callback:86] F1 micro: 98.82
[NeMo I 2020-05-22 17:13:49 token_classification_callback:89] precision    recall  f1-score   support
    
    O (label id: 0)     0.9938    0.9957    0.9947     22092
    B (label id: 1)     0.8843    0.9034    0.8938       787
    I (label id: 2)     0.9505    0.8982    0.9236      1090
    
           accuracy                         0.9882     23969
          macro avg     0.9429    0.9324    0.9374     23969
       weighted avg     0.9882    0.9882    0.9882     23969
```